# First example

In [1]:
# EXECUTE THIS CELL ONLY THE FIRST TIME!!
import os
os.chdir("..")

In [2]:
from synthDataGen.controller import DataControllerESIOS

dc = DataControllerESIOS("inputParams.json", "./synthDataGen/settings/")

dc.initialYear

2007

In [ ]:
import os
os.getcwd()

In [ ]:
a = None
if not a:
    print("hey")

In [ ]:
import pandas as pd
mes = pd.date_range('01-1972', '03-1972', freq='H')
datos = list(range(1, len(mes) + 1, 1))
datos2 = [i*2 for i in datos]

df = pd.DataFrame(list(zip(mes, datos, datos2)), columns=['mes', 'uno', 'dos'])
df.set_index(['mes'], inplace=True)

df.head(20)

In [ ]:
primerValor = df.index[0]
ultimoValor = df.index[-1] 
sampleoCHAR = str(90) + 'min'
sampleoDeseado = pd.date_range(primerValor, ultimoValor, freq=sampleoCHAR)

print(str(primerValor) + " " + str(ultimoValor) + " " + (sampleoCHAR) + " " + str(sampleoDeseado))

In [ ]:
df = df.reindex(df.index.union(sampleoDeseado))
df.head(20)

In [ ]:
df = df.interpolate(method="linear", order=2)
df.head(20)

# Second example

In [ ]:
from __future__ import division
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
 

from bibliotecaEsios import BajadaDatosESIOS
import bibliotecaGeneral as zsan

In [ ]:
try:
   from ficheroDeClaves import CLAVEdeESIOS
   id_SFP = CLAVEdeESIOS
except:
    cadenaError = '\n '
    cadenaError = '\n Con objeto de que el código sea lo más "privado" posible'
    cadenaError += '\n para ser ejecutado en modo demo '
    cadenaError += '\n crear un fichero llamado ficheroDeClaves.py'
    cadenaError += '\n y dentro del mismo definir la variable CLAVEdeESIOS = el ID que se posea para consultas ESIOS'
    raise Exception(cadenaError) 


In [ ]:
horasAdelante = 10
frecuenciaSampleoMinutos = 15


AGNOINICIALCALCULOS = 2007

diccionarioAgnoIPC = {   # EN TANTO POR CIENTO
    "2022": 10,
    "2021": 10,    
    "2020": 10,    
    "2019": 10,    
    "2018": 10,    
    "2017": 10,    
    "2016": 10,
    "2015": 10,    
    "2014": 10,
    "2013": 10,    
    "2012": 10,    
    "2011": 10,
    "2010": 10,
    "2009": 10,
    "2008": 10,    
    "2007": 10    
       }

# A revisión
time_trunc = 'hours'  # Este es el parámetro de llamada de la función encontrada para interacción ESIOS
ordenSpline = 2

In [ ]:
# Pseudoconstantes
ahora = datetime.now()
minutoDesde = ahora.minute
horaDesde  = ahora.hour
diaDesde  = ahora.day
mesDesde  = ahora.month
agnoActual = ahora.year

listaAgnos = list(range(AGNOINICIALCALCULOS, agnoActual))

In [ ]:
# Primera columna del dataframe de salida
inicio = datetime(AGNOINICIALCALCULOS, mesDesde, diaDesde, horaDesde, minutoDesde)       
hasta  = inicio + timedelta(hours=horasAdelante)

print("inicio: " + str(inicio) + " - hasta: " + str(hasta))

a =  BajadaDatosESIOS(id_SFP)
df = a.dataframe_lista_de_indicadores_de_esios_por_fechas([10211], inicio, hasta, time_trunc=time_trunc,
                                                         inlcuye29DeFebrero = '')
df = df.filter(['value'], axis=1)

df = df.rename(columns={'value': str(AGNOINICIALCALCULOS)})
df

In [ ]:
### 2 de 2: Cambiamos el año del indice, desde el primer año de la serie histórica al año acutal
df.index = df.index + pd.offsets.DateOffset(years=agnoActual - AGNOINICIALCALCULOS)
df

In [ ]:
for agno in range(AGNOINICIALCALCULOS + 1, agnoActual):
    inicio = datetime(agno, mesDesde, diaDesde, horaDesde, minutoDesde)      # desde comienzo del día
    hasta  = inicio + timedelta(hours=horasAdelante)
    aux = a.dataframe_lista_de_indicadores_de_esios_por_fechas([10211], inicio, hasta, 
                                                        time_trunc=time_trunc, inlcuye29DeFebrero = '') 
    df[str(agno)] = list(aux['value'])

In [ ]:
dicionarioCorreccionIPC = zsan.calculaCorrecionDePreciosPorIPC(diccionarioAgnoIPC)
dicionarioCorreccionIPC

In [ ]:
# Con los objetos que estamos manejando, la operación es trivial
for elemento in df.columns:
    df[elemento] = df[elemento] * dicionarioCorreccionIPC[elemento]
df

In [ ]:
type(min(["2004", "1998"]))

In [ ]:
print('DataFrame se resamplea a ', frecuenciaSampleoMinutos, ' minutos')
df = zsan.resampleaDataFrame(df, frecuenciaSampleoMinutos, method='polynomial', order=ordenSpline)

In [ ]:
df.head(10)